## EECS 453/551
# Video background subtraction using SVD

In this exercise we will discover how the SVD can magically estimate the background in a video even when there is a lot of noise and missing data.
___

## Before we begin

Run the following code cell by clicking it and either

* choosing Cell -> Run in the toolbar
* Pressing Ctrl+Enter

This will install and load the Julia packages we need for today.

In [ ]:
# install packages if not already present:
Pkg.add("MAT")      # for loading .mat files
Pkg.add("Interact") # for interacting with plots
Pkg.add("Images")

# load packages:
using MAT, Interact, Reactive, Images #, PyPlot
import Images.imresize

# choose between lobby and xylophone video footage:
lobby = permutedims(matread("lobby.mat")["MovMat"],[2;1;3]);
xylophone = permutedims(matread("xylophone.mat")["MovMat"],[2;1;3]);
scale!(1/maximum(xylophone),xylophone);

You may wish to take a few minutes to familiarize yourself with the notebook interface. Choose Help -> User Interface Tour for a brief introduction.

In Julia, you can type ? followed by any object (variable, function, module) to get its description. Try typing `?MAT` into a new code cell and running it.

## Objective

The objective of this exercise is to take as an input the video you have just seen and return as an output the background (i.e. without the people moving for the lobby video).

### Task 1

Find in the function below the portion where the variable `backgroundVec` is computed. Note how the SVD is used to compute it. Examine how the `residualVec` variable is computed and note that

    footageVec = backgroundVec + residualVec

Hence if we successfully are able to estimate the background, `residualVec` should contain:

1. People moving around
2. Nothing (empty)

**Do you expect 1. or 2. to be true? Why?**

Now **run the cell** to add `process()` to your workspace.

In [ ]:
"""
    process(footage) -> background,residual
Use SVD to separate input footage into background (static objects)
and residual (fleeting objects).
"""
function process(footage)
    (m,n,numFrames) = size(footage)
    
    # Each movie frame is a 128x160 array.
    # Reshape each frame into a single column vector
    # to ease manipulation:
    footageVec = reshape(footage,m*n,numFrames)
    
    ###################################
    # Process using SVD
    r = 1 # rank of "background" is 1

    # U,S,V will contain first r components of SVD:
    (U,S,V) = svds(footageVec,nsv=r)
    
    # take first r components as background
    backgroundVec = (U*diagm(S))*V'
    
    # calculate residual
    residualVec = footageVec - backgroundVec    
    ###################################

    # reshape background and residual into footage dimensions
    background = reshape(backgroundVec,size(footage))
    residual = reshape(residualVec,size(footage))
    
    return background,residual
end

### Task 2

Run the following code cell. You should see a slider above a plot with two subplots. On the left is the original footage; on the right is the background as estimated by SVD.

**Is it working?**

In [ ]:
footage = lobby

# perform SVD-based background subtraction:
background,residual = process(footage)

### display results
m,n,numFrames = size(footage)
ws = ones(5,n,numFrames) # whitespace
img = grayim([footage;ws;background])

@manipulate for s = slider(1.0:0.2:3.0, label="Scale factor"),
    frame = slider(1:numFrames, label="Frame")
    
    # select frame
    i = sliceim(img,"z",frame)
    
    # scale
    imresize(i,tuple([round(Int64,s*dim) for dim in size(i)]...))
    
end

### Task 3: adding noise

The movie you just processed using SVD was relatively noise-free. Let us start adding noise and removing entries and see how well SVD does.

Study the function `addnoise()` in the following cell. **What is the code doing mathematically?** Now **run the cell** to add the function to your workspace.

Enter a value of 0.1 for $ \sigma $ in the next cell down, then run it.

**Do you see the movie get noisier? Is the SVD able to extract the background?**

* Now repeat for $ \sigma = 0.5 $, $ \sigma = 1 $ and $ \sigma = 2 $. **How does the SVD do?**

In [ ]:
"""
Add noise to footage.

σ is noise standard deviation.

(type symbol with "\sigma<TAB>")
"""
function addnoise(footage,σ)
    # Interpret this mathematically!
    footage = (footage + σ*rand(size(footage)))
    return footage
end   

In [ ]:
footage = lobby

############################
σ =  # Enter your value here

# Add noise:
footage = addnoise(footage,σ)
############################

# perform background subtraction on noisy footage:
background,residual = process(footage)

### display results:
m,n,numFrames = size(footage)
ws = ones(5,n,numFrames) # whitespace
img = grayim([footage;ws;background])
@manipulate for s = slider(1.0:0.2:3.0, label="Scale image"),
    frame = slider(1:numFrames, label="Frame")
    
    i = sliceim(img,"z",frame)
    imresize(i,tuple([round(Int64,s*dim) for dim in size(i)]...))
end

### Task 4: removing entries

Now we will corrupt the data by randomly removing a portion of the entries.

Study `corrupt()` in the cell below. **What is it doing to the input footage?** Now **run the cell** to add `corrupt()` to your workspace.

Enter a value of 0.9 for `p` in the next cell down, then run it.

**Does the movie have missing entries? Is the SVD successfully separating the background?**

Now repeat for `p = 0.75`, `p = 0.5`, `p = 0.25`, `p = 0.1`. **Do you see the movie have more blank entries? Is the SVD still working?**

In [ ]:
"""
Corrupt footage by removing entries.

Each entry is removed with probability `p`.
"""
function corrupt(footage,p)
    if p < 0.0 || p > 1.0
        error("p must be in [0,1]")
    end
    # Interpret this mathematically!
    # Type ?rand if you aren't sure what rand() does.
    mask = rand(size(footage)) .< p

    # Mask some entries
    footage = footage.*mask
    return footage
end

In [ ]:
footage = lobby

# Add noise:
σ = 0.1
footage = addnoise(footage,σ)

############################
p =  # Enter your value here

# Delete entries
footage = corrupt(footage,p)
############################

# Process footage using SVD
background,res = process(footage)

# Display
m,n,numFrames = size(footage)
ws = ones(5,n,numFrames) # whitespace
img = grayim([footage;ws;background])
@manipulate for s = slider(1.0:0.5:3.0, label="Scale factor"),
    frame = slider(1:numFrames, label="Frame")
    
    i = sliceim(img,"z",frame)
    imresize(i,tuple([round(Int64,s*dim) for dim in size(i)]...))
end    

The corrupted footage is on the left; the background (first component of SVD) is on the right.

## Task 5

What if we considered the first two SVD components to be the background rather than just one? What happens as we change $\sigma$ and p?

The following method returns both rank-1 and rank-2 backgrounds and allows us to quickly change $\sigma$ and p. **Run the cell** to add this method to your workspace.

In [ ]:
"""
    process(footage) -> background,residual
Use SVD to separate input footage into background (static objects)
and residual (fleeting objects).
"""
function process(footage,σ,p)
    
    (m,n,numFrames) = size(footage)
    footage = addnoise(footage,σ) # Add noise:
    footage = corrupt(footage,p) # Delete entries:
    footageVec = reshape(footage,m*n,numFrames) # Reshape each frame into a column
    
    (U,S,V) = svds(footageVec,nsv=2) # Process using SVD
    
    backgroundVec = (U[:,1]*diagm(S)[1])*V[:,1]'# take first 1 components as background
    backgroundVec2 = (U*diagm(S))*V'            # take first 2 components as background2
    residualVec = footageVec - backgroundVec    # calculate residual
    residualVec2 = footageVec - backgroundVec2  # calculate residual2
    
    # reshape footage dimensions
    background  = reshape(backgroundVec,size(footage))
    background2 = reshape(backgroundVec2,size(footage))
    
    return footage,background,background2
end

Now we can illustrate the difference between rank-1 and rank-2 background separation. **Run the following cell** to see footage on the left, rank-1 background in the middle, and rank-2 background on the right.

In [ ]:
###########################
# Enter your values here
σ = 
p = 
footage = lobby # xylophone
###########################

# perform background subtraction
fg,bg,bg2 = process(footage,σ,p)

# display results
m,n,numFrames = size(footage)
ws = ones(5,n,numFrames) # whitespace
img = grayim([fg;ws;bg;ws;bg2])
@manipulate for s = slider(1.0:0.2:2.0, label="Scale factor"),
    frame = slider(1:numFrames, label="Frame")

    i = sliceim(img,"z",frame)
    imresize(i,tuple([round(Int64,s*dim) for dim in size(i)]...))    
end

### Task 6

In the cell above, change 
```julia
footage = lobby
```
to 
```julia
footage = xylophone
```
and run the cell again. **Does SVD-based processing still work?**